In [19]:
import os
import yaml
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pathlib import Path
import json
from sklearn.metrics import confusion_matrix
import seaborn as sns

class SixRayYOLOv8Detector:
    def __init__(self, dataset_path="D:/NOT/sixray_yolov8.v964i.yolov8"):
        self.dataset_path = Path(dataset_path)
        self.model = None
        self.class_names = ['Gun', 'Knife', 'Wrench', 'Pliers', 'Scissors']
        self.results_dir = Path("sixray_results")
        self.results_dir.mkdir(exist_ok=True)
        
        # Veri seti yapısını kontrol et
        self.check_dataset_structure()
        
    def check_dataset_structure(self):
        """Veri seti yapısını kontrol et ve rapor ver"""
        print("📁 Veri seti yapısı kontrol ediliyor...")
        print(f"Dataset yolu: {self.dataset_path}")
        
        # Ana klasörleri kontrol et
        train_dir = self.dataset_path / "train"
        valid_dir = self.dataset_path / "valid" 
        test_dir = self.dataset_path / "test"
        
        # data.yaml dosyasını kontrol et
        yaml_files = list(self.dataset_path.glob("*.yaml"))
        if yaml_files:
            self.yaml_path = yaml_files[0]
            print(f"✅ YAML config dosyası bulundu: {self.yaml_path.name}")
        else:
            print("❌ YAML config dosyası bulunamadı!")
            self.create_yaml_config()
            
        # Klasör yapısını göster
        for split_name, split_dir in [("Train", train_dir), ("Valid", valid_dir), ("Test", test_dir)]:
            if split_dir.exists():
                images_dir = split_dir / "images"
                labels_dir = split_dir / "labels"
                
                if images_dir.exists() and labels_dir.exists():
                    img_count = len(list(images_dir.glob("*.jpg"))) + len(list(images_dir.glob("*.png")))
                    label_count = len(list(labels_dir.glob("*.txt")))
                    print(f"✅ {split_name}: {img_count} görüntü, {label_count} etiket")
                else:
                    print(f"❌ {split_name}: images veya labels klasörü eksik")
    
    def create_yaml_config(self):
        """YAML config dosyası oluştur"""
        config = {
            'path': str(self.dataset_path.absolute()),
            'train': 'train/images',
            'val': 'valid/images', 
            'test': 'test/images',
            'nc': 5,  # class sayısı
            'names': self.class_names
        }
        
        self.yaml_path = self.dataset_path / "data.yaml"
        with open(self.yaml_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        
        print(f"✅ YAML config dosyası oluşturuldu: {self.yaml_path}")
    
    def load_pretrained_model(self, model_size='n'):
        """Pre-trained YOLOv8 modelini yükle"""
        model_name = f'yolov8{model_size}.pt'
        self.model = YOLO(model_name)
        print(f"✅ {model_name} modeli yüklendi")
        print(f"📊 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
        
    def train_model(self, epochs=100, batch_size=16, img_size=640, model_size='n'):
        """Modeli eğit"""
        if self.model is None:
            self.load_pretrained_model(model_size)
        
        print(f"🚀 Model eğitimi başlıyor...")
        print(f"   Epochs: {epochs}")
        print(f"   Batch Size: {batch_size}")
        print(f"   Image Size: {img_size}")
        
        # Eğitim parametreleri
        results = self.model.train(
            data=str(self.yaml_path),
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            project=str(self.results_dir),
            name='sixray_training',
            save=True,
            plots=True,
            val=True,
            patience=20,  # Early stopping
            save_period=10,  # Her 10 epoch'ta kaydet
            cache=True,  # Veri setini RAM'e cache'le
            workers=4
        )
        
        print("✅ Eğitim tamamlandı!")
        return results
    
    def load_trained_model(self, model_path=None):
        """Eğitilmiş modeli yükle"""
        if model_path is None:
            # En son eğitilen modeli bul
            best_path = self.results_dir / "sixray_training" / "weights" / "best.pt"
            if best_path.exists():
                model_path = best_path
            else:
                print("❌ Eğitilmiş model bulunamadı!")
                return False
        
        self.model = YOLO(str(model_path))
        print(f"✅ Eğitilmiş model yüklendi: {model_path}")
        return True
    
    def predict_single_image(self, image_path, conf_threshold=0.25, save_result=True):
        """Tek görüntü üzerinde tahmin yap"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return None
        
        # Tahmin yap
        results = self.model(image_path, conf=conf_threshold)
        
        if save_result:
            # Sonuçları görselleştir ve kaydet
            annotated_img = results[0].plot()
            
            # Dosya adını al
            img_name = Path(image_path).stem
            save_path = self.results_dir / f"prediction_{img_name}.jpg"
            
            # Kaydet
            cv2.imwrite(str(save_path), annotated_img)
            print(f"📸 Sonuç kaydedildi: {save_path}")
        
        return results[0]
    
    def predict_batch(self, image_folder, conf_threshold=0.25):
        """Bir klasördeki tüm görüntüleri işle"""
        image_folder = Path(image_folder)
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
        
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(image_folder.glob(ext)))
        
        print(f"📂 {len(image_files)} görüntü bulundu")
        
        all_results = []
        for img_path in image_files:
            print(f"🔍 İşleniyor: {img_path.name}")
            result = self.predict_single_image(img_path, conf_threshold)
            if result:
                all_results.append(result)
        
        return all_results
    
    def evaluate_model(self):
        """Modeli test seti üzerinde değerlendir"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return None
        
        print("📊 Model değerlendirmesi başlıyor...")
        
        # Validasyon yapı
        results = self.model.val(data=str(self.yaml_path))
        
        # Metrikleri yazdır
        print(f"\n📈 Değerlendirme Sonuçları:")
        print(f"   mAP@0.5: {results.box.map50:.4f}")
        print(f"   mAP@0.5:0.95: {results.box.map:.4f}")
        print(f"   Precision: {results.box.mp:.4f}")
        print(f"   Recall: {results.box.mr:.4f}")
        
        # Sınıf bazında sonuçlar
        if hasattr(results.box, 'ap_class_index'):
            print(f"\n🎯 Sınıf Bazında mAP@0.5:")
            for i, class_name in enumerate(self.class_names):
                if i < len(results.box.ap50):
                    print(f"   {class_name}: {results.box.ap50[i]:.4f}")
        
        return results
    
    def visualize_predictions(self, image_path, conf_threshold=0.25):
        """Tahminleri matplotlib ile görselleştir"""
        # Tahmin yap
        results = self.model(image_path, conf=conf_threshold)
        result = results[0]
        
        # Görüntüyü yükle
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Plot oluştur
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        # Orijinal görüntü
        ax1.imshow(image)
        ax1.set_title('Orijinal Görüntü', fontsize=16)
        ax1.axis('off')
        
        # Tahmin sonuçları
        annotated = result.plot()
        annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        ax2.imshow(annotated)
        ax2.set_title(f'YOLOv8 Tahminleri (conf≥{conf_threshold})', fontsize=16)
        ax2.axis('off')
        
        # Deteksiyon bilgilerini yazdır
        if result.boxes is not None and len(result.boxes) > 0:
            print(f"\n🎯 Tespit edilen objeler ({len(result.boxes)} adet):")
            for i, box in enumerate(result.boxes):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = self.class_names[class_id] if class_id < len(self.class_names) else f"Class_{class_id}"
                print(f"   {i+1}. {class_name}: {confidence:.3f}")
        else:
            print("❌ Hiç obje tespit edilmedi!")
        
        plt.tight_layout()
        plt.show()
        
        return result
    
    def create_detection_report(self, test_folder=None):
        """Detaylı test raporu oluştur"""
        if test_folder is None:
            test_folder = self.dataset_path / "test" / "images"
        
        if not test_folder.exists():
            print(f"❌ Test klasörü bulunamadı: {test_folder}")
            return
        
        print("📋 Detaylı test raporu oluşturuluyor...")
        
        # Test görüntülerini al
        image_files = list(test_folder.glob("*.jpg")) + list(test_folder.glob("*.png"))
        
        total_detections = 0
        class_detections = {name: 0 for name in self.class_names}
        confidence_scores = []
        
        for img_path in image_files[:20]:  # İlk 20 görüntü için
            results = self.model(str(img_path), conf=0.25)
            result = results[0]
            
            if result.boxes is not None:
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    
                    if class_id < len(self.class_names):
                        class_name = self.class_names[class_id]
                        class_detections[class_name] += 1
                        confidence_scores.append(confidence)
                        total_detections += 1
        
        # Raporu yazdır
        print(f"\n📊 Test Raporu:")
        print(f"   Toplam tespit: {total_detections}")
        print(f"   Ortalama güven skoru: {np.mean(confidence_scores):.3f}")
        print(f"\n🎯 Sınıf bazında tespitler:")
        for class_name, count in class_detections.items():
            percentage = (count / total_detections * 100) if total_detections > 0 else 0
            print(f"   {class_name}: {count} ({percentage:.1f}%)")
    
    def live_detection_demo(self):
        """Canlı kamera ile detection demosu"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return
        
        print("📹 Canlı detection başlıyor... (ESC ile çıkış)")
        
        cap = cv2.VideoCapture(0)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # YOLOv8 ile tahmin
            results = self.model(frame, conf=0.3, verbose=False)
            
            # Sonuçları çiz
            annotated_frame = results[0].plot()
            
            # Göster
            cv2.imshow('SixRay YOLOv8 Live Detection', annotated_frame)
            
            # ESC ile çıkış
            if cv2.waitKey(1) & 0xFF == 27:
                break
        
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Canlı detection sonlandırıldı")

# Ana kullanım fonksiyonu
def main():
    # Detector oluştur
    detector = SixRayYOLOv8Detector()
    
    print("🎯 SixRay YOLOv8 Object Detection")
    print("=" * 50)
    
    while True:
        print("\n📋 Menü:")
        print("1. Model eğit")
        print("2. Eğitilmiş model yükle")
        print("3. Tek görüntü test et")
        print("4. Klasör test et")
        print("5. Model değerlendir")
        print("6. Test raporu oluştur")
        print("7. Canlı detection demo")
        print("0. Çıkış")
        
        choice = input("\nSeçiminiz (0-7): ").strip()
        
        if choice == "1":
            epochs = int(input("Epoch sayısı (varsayılan 100): ") or 100)
            batch_size = int(input("Batch size (varsayılan 16): ") or 16)
            model_size = input("Model boyutu (n/s/m/l/x, varsayılan n): ").strip() or 'n'
            
            detector.train_model(epochs=epochs, batch_size=batch_size, model_size=model_size)
            
        elif choice == "2":
            model_path = input("Model yolu (boş bırakın en son eğitileni kullanmak için): ").strip()
            if not model_path:
                model_path = None
            detector.load_trained_model(model_path)
            
        elif choice == "3":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            
            img_path = input("Görüntü yolu girin: ").strip()
            if os.path.exists(img_path):
                detector.visualize_predictions(img_path)
            else:
                print("❌ Dosya bulunamadı!")
                
        elif choice == "4":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
                
            folder_path = input("Klasör yolu girin: ").strip()
            if os.path.exists(folder_path):
                detector.predict_batch(folder_path)
            else:
                print("❌ Klasör bulunamadı!")
                
        elif choice == "5":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.evaluate_model()
            
        elif choice == "6":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.create_detection_report()
            
        elif choice == "7":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.live_detection_demo()
            
        elif choice == "0":
            print("👋 Görüşmek üzere!")
            break
            
        else:
            print("❌ Geçersiz seçim!")

if __name__ == "__main__":
    main()

📁 Veri seti yapısı kontrol ediliyor...
Dataset yolu: D:\NOT\sixray_yolov8.v964i.yolov8
✅ YAML config dosyası bulundu: data.yaml
✅ Train: 5806 görüntü, 5806 etiket
✅ Valid: 1628 görüntü, 1628 etiket
✅ Test: 829 görüntü, 829 etiket
🎯 SixRay YOLOv8 Object Detection

📋 Menü:
1. Model eğit
2. Eğitilmiş model yükle
3. Tek görüntü test et
4. Klasör test et
5. Model değerlendir
6. Test raporu oluştur
7. Canlı detection demo
0. Çıkış

Seçiminiz (0-7): 1
Epoch sayısı (varsayılan 100): 60
Batch size (varsayılan 16): 16
Model boyutu (n/s/m/l/x, varsayılan n): n
✅ yolov8n.pt modeli yüklendi
📊 Device: CUDA
🚀 Model eğitimi başlıyor...
   Epochs: 60
   Batch Size: 16
   Image Size: 640
New https://pypi.org/project/ultralytics/8.3.152 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.145  Python-3.11.5 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=

train: Scanning D:\NOT\sixray_yolov8.v964i.yolov8\train\labels.cache... 5806 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5806/5806 [00:00<?, ?it/s]

WARNING train: 10.0GB RAM required to cache images with 50% safety margin but only 2.6/15.9GB available, not caching images
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access  (ping: 0.20.1 ms, read: 194.485.0 MB/s, size: 38.8 KB)


val: Scanning D:\NOT\sixray_yolov8.v964i.yolov8\valid\labels.cache... 1628 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1628/1628 [00:00<?, ?it/s]


...

                   all       1628       3049      0.916      0.843      0.904      0.677
                  Gun-        524        835      0.964       0.95      0.982       0.78
                Knife-        259        368      0.923      0.823      0.877      0.658
               Pliers-        794       1110      0.926      0.831      0.911      0.672
             Scissors-        178        206      0.889       0.85      0.903      0.648
               Wrench-        392        530      0.877       0.76      0.848      0.628
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to sixray_results\sixray_training5
✅ Eğitim tamamlandı!

📋 Menü:
1. Model eğit
2. Eğitilmiş model yükle
3. Tek görüntü test et
4. Klasör test et
5. Model değerlendir
6. Test raporu oluştur
7. Canlı detection demo
0. Çıkış

Seçiminiz (0-7): 5
📊 Model değerlendirmesi başlıyor...
Ultralytics 8.3.145  Python-3.11.5 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Mod

val: Scanning D:\NOT\sixray_yolov8.v964i.yolov8\valid\labels.cache... 1628 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1628/1628 [00:00<?, ?it/s]


WARNING val: 2.8GB RAM required to cache images with 50% safety margin but only 2.2/15.9GB available, not caching images


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 102/102 [00:14<00:00,  6.82it/s]


                   all       1628       3049      0.916      0.842      0.904      0.676
                  Gun-        524        835      0.963       0.95      0.982       0.78
                Knife-        259        368      0.924      0.823      0.877      0.657
               Pliers-        794       1110      0.927       0.83      0.911      0.672
             Scissors-        178        206       0.89       0.85      0.904      0.645
               Wrench-        392        530      0.878       0.76      0.848      0.627
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to sixray_results\sixray_training52

📈 Değerlendirme Sonuçları:
   mAP@0.5: 0.9045
   mAP@0.5:0.95: 0.6762
   Precision: 0.9163
   Recall: 0.8424

🎯 Sınıf Bazında mAP@0.5:
   Gun: 0.9821
   Knife: 0.8774
   Wrench: 0.9108
   Pliers: 0.9038
   Scissors: 0.8481

📋 Menü:
1. Model eğit
2. Eğitilmiş model yükle
3. Tek görüntü test et
4. Klasör test et
5. Model değerlendir


In [1]:
import os
import yaml
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pathlib import Path
import json
from sklearn.metrics import confusion_matrix
import seaborn as sns

class SixRayYOLOv8Detector:
    def __init__(self, dataset_path="D:/NOT/sixray_yolov8.v964i.yolov8"):
        self.dataset_path = Path(dataset_path)
        self.model = None
        self.class_names = ['Gun', 'Knife', 'Wrench', 'Pliers', 'Scissors']
        self.results_dir = Path("sixray_results")
        self.results_dir.mkdir(exist_ok=True)
        
        # Veri seti yapısını kontrol et
        self.check_dataset_structure()
        
    def check_dataset_structure(self):
        """Veri seti yapısını kontrol et ve rapor ver"""
        print("📁 Veri seti yapısı kontrol ediliyor...")
        print(f"Dataset yolu: {self.dataset_path}")
        
        # Ana klasörleri kontrol et
        train_dir = self.dataset_path / "train"
        valid_dir = self.dataset_path / "valid" 
        test_dir = self.dataset_path / "test"
        
        # data.yaml dosyasını kontrol et
        yaml_files = list(self.dataset_path.glob("*.yaml"))
        if yaml_files:
            self.yaml_path = yaml_files[0]
            print(f"✅ YAML config dosyası bulundu: {self.yaml_path.name}")
        else:
            print("❌ YAML config dosyası bulunamadı!")
            self.create_yaml_config()
            
        # Klasör yapısını göster
        for split_name, split_dir in [("Train", train_dir), ("Valid", valid_dir), ("Test", test_dir)]:
            if split_dir.exists():
                images_dir = split_dir / "images"
                labels_dir = split_dir / "labels"
                
                if images_dir.exists() and labels_dir.exists():
                    img_count = len(list(images_dir.glob("*.jpg"))) + len(list(images_dir.glob("*.png")))
                    label_count = len(list(labels_dir.glob("*.txt")))
                    print(f"✅ {split_name}: {img_count} görüntü, {label_count} etiket")
                else:
                    print(f"❌ {split_name}: images veya labels klasörü eksik")
    
    def create_yaml_config(self):
        """YAML config dosyası oluştur"""
        config = {
            'path': str(self.dataset_path.absolute()),
            'train': 'train/images',
            'val': 'valid/images', 
            'test': 'test/images',
            'nc': 5,  # class sayısı
            'names': self.class_names
        }
        
        self.yaml_path = self.dataset_path / "data.yaml"
        with open(self.yaml_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        
        print(f"✅ YAML config dosyası oluşturuldu: {self.yaml_path}")
    
    def load_pretrained_model(self, model_size='n'):
        """Pre-trained YOLOv8 modelini yükle"""
        model_name = f'yolov8{model_size}.pt'
        self.model = YOLO(model_name)
        print(f"✅ {model_name} modeli yüklendi")
        print(f"📊 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
        
    def train_model(self, epochs=100, batch_size=16, img_size=640, model_size='n'):
        """Modeli eğit"""
        if self.model is None:
            self.load_pretrained_model(model_size)
        
        print(f"🚀 Model eğitimi başlıyor...")
        print(f"   Epochs: {epochs}")
        print(f"   Batch Size: {batch_size}")
        print(f"   Image Size: {img_size}")
        
        # Eğitim parametreleri
        results = self.model.train(
            data=str(self.yaml_path),
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            project=str(self.results_dir),
            name='sixray_training',
            save=True,
            plots=True,
            val=True,
            patience=20,  # Early stopping
            save_period=10,  # Her 10 epoch'ta kaydet
            cache=True,  # Veri setini RAM'e cache'le
            workers=4
        )
        
        print("✅ Eğitim tamamlandı!")
        return results
    
    def load_trained_model(self, model_path=None):
        """Eğitilmiş modeli yükle"""
        if model_path is None:
            # En son eğitilen modeli bul
            best_path = self.results_dir / "sixray_training" / "weights" / "best.pt"
            if best_path.exists():
                model_path = best_path
            else:
                print("❌ Eğitilmiş model bulunamadı!")
                return False
        
        self.model = YOLO(str(model_path))
        print(f"✅ Eğitilmiş model yüklendi: {model_path}")
        return True
    
    def predict_single_image(self, image_path, conf_threshold=0.25, save_result=True):
        """Tek görüntü üzerinde tahmin yap"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return None
        
        # Tahmin yap
        results = self.model(image_path, conf=conf_threshold)
        
        if save_result:
            # Sonuçları görselleştir ve kaydet
            annotated_img = results[0].plot()
            
            # Dosya adını al
            img_name = Path(image_path).stem
            save_path = self.results_dir / f"prediction_{img_name}.jpg"
            
            # Kaydet
            cv2.imwrite(str(save_path), annotated_img)
            print(f"📸 Sonuç kaydedildi: {save_path}")
        
        return results[0]
    
    def predict_batch(self, image_folder, conf_threshold=0.25):
        """Bir klasördeki tüm görüntüleri işle"""
        image_folder = Path(image_folder)
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
        
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(image_folder.glob(ext)))
        
        print(f"📂 {len(image_files)} görüntü bulundu")
        
        all_results = []
        for img_path in image_files:
            print(f"🔍 İşleniyor: {img_path.name}")
            result = self.predict_single_image(img_path, conf_threshold)
            if result:
                all_results.append(result)
        
        return all_results
    
    def evaluate_model(self):
        """Modeli test seti üzerinde değerlendir"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return None
        
        print("📊 Model değerlendirmesi başlıyor...")
        
        # Validasyon yapı
        results = self.model.val(data=str(self.yaml_path))
        
        # Metrikleri yazdır
        print(f"\n📈 Değerlendirme Sonuçları:")
        print(f"   mAP@0.5: {results.box.map50:.4f}")
        print(f"   mAP@0.5:0.95: {results.box.map:.4f}")
        print(f"   Precision: {results.box.mp:.4f}")
        print(f"   Recall: {results.box.mr:.4f}")
        
        # Sınıf bazında sonuçlar
        if hasattr(results.box, 'ap_class_index'):
            print(f"\n🎯 Sınıf Bazında mAP@0.5:")
            for i, class_name in enumerate(self.class_names):
                if i < len(results.box.ap50):
                    print(f"   {class_name}: {results.box.ap50[i]:.4f}")
        
        return results
    
    def visualize_predictions(self, image_path, conf_threshold=0.25):
        """Tahminleri matplotlib ile görselleştir"""
        # Tahmin yap
        results = self.model(image_path, conf=conf_threshold)
        result = results[0]
        
        # Görüntüyü yükle
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Plot oluştur
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        # Orijinal görüntü
        ax1.imshow(image)
        ax1.set_title('Orijinal Görüntü', fontsize=16)
        ax1.axis('off')
        
        # Tahmin sonuçları
        annotated = result.plot()
        annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        ax2.imshow(annotated)
        ax2.set_title(f'YOLOv8 Tahminleri (conf≥{conf_threshold})', fontsize=16)
        ax2.axis('off')
        
        # Deteksiyon bilgilerini yazdır
        if result.boxes is not None and len(result.boxes) > 0:
            print(f"\n🎯 Tespit edilen objeler ({len(result.boxes)} adet):")
            for i, box in enumerate(result.boxes):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = self.class_names[class_id] if class_id < len(self.class_names) else f"Class_{class_id}"
                print(f"   {i+1}. {class_name}: {confidence:.3f}")
        else:
            print("❌ Hiç obje tespit edilmedi!")
        
        plt.tight_layout()
        plt.show()
        
        return result
    
    def create_detection_report(self, test_folder=None):
        """Detaylı test raporu oluştur"""
        if test_folder is None:
            test_folder = self.dataset_path / "test" / "images"
        
        if not test_folder.exists():
            print(f"❌ Test klasörü bulunamadı: {test_folder}")
            return
        
        print("📋 Detaylı test raporu oluşturuluyor...")
        
        # Test görüntülerini al
        image_files = list(test_folder.glob("*.jpg")) + list(test_folder.glob("*.png"))
        
        total_detections = 0
        class_detections = {name: 0 for name in self.class_names}
        confidence_scores = []
        
        for img_path in image_files[:20]:  # İlk 20 görüntü için
            results = self.model(str(img_path), conf=0.25)
            result = results[0]
            
            if result.boxes is not None:
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    
                    if class_id < len(self.class_names):
                        class_name = self.class_names[class_id]
                        class_detections[class_name] += 1
                        confidence_scores.append(confidence)
                        total_detections += 1
        
        # Raporu yazdır
        print(f"\n📊 Test Raporu:")
        print(f"   Toplam tespit: {total_detections}")
        print(f"   Ortalama güven skoru: {np.mean(confidence_scores):.3f}")
        print(f"\n🎯 Sınıf bazında tespitler:")
        for class_name, count in class_detections.items():
            percentage = (count / total_detections * 100) if total_detections > 0 else 0
            print(f"   {class_name}: {count} ({percentage:.1f}%)")
    
    def live_detection_demo(self):
        """Canlı kamera ile detection demosu"""
        if self.model is None:
            print("❌ Model henüz yüklenmedi!")
            return
        
        print("📹 Canlı detection başlıyor... (ESC ile çıkış)")
        
        cap = cv2.VideoCapture(0)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # YOLOv8 ile tahmin
            results = self.model(frame, conf=0.3, verbose=False)
            
            # Sonuçları çiz
            annotated_frame = results[0].plot()
            
            # Göster
            cv2.imshow('SixRay YOLOv8 Live Detection', annotated_frame)
            
            # ESC ile çıkış
            if cv2.waitKey(1) & 0xFF == 27:
                break
        
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Canlı detection sonlandırıldı")

# Ana kullanım fonksiyonu
def main():
    # Detector oluştur
    detector = SixRayYOLOv8Detector()
    
    print("🎯 SixRay YOLOv8 Object Detection")
    print("=" * 50)
    
    while True:
        print("\n📋 Menü:")
        print("1. Model eğit")
        print("2. Eğitilmiş model yükle")
        print("3. Tek görüntü test et")
        print("4. Klasör test et")
        print("5. Model değerlendir")
        print("6. Test raporu oluştur")
        print("7. Canlı detection demo")
        print("0. Çıkış")
        
        choice = input("\nSeçiminiz (0-7): ").strip()
        
        if choice == "1":
            epochs = int(input("Epoch sayısı (varsayılan 100): ") or 200)
            batch_size = int(input("Batch size (varsayılan 16): ") or 16)
            model_size = input("Model boyutu (n/s/m/l/x, varsayılan n): ").strip() or 'n'
            
            detector.train_model(epochs=epochs, batch_size=batch_size, model_size=model_size)
            
        elif choice == "2":
            model_path = input("Model yolu (boş bırakın en son eğitileni kullanmak için): ").strip()
            if not model_path:
                model_path = None
            detector.load_trained_model(model_path)
            
        elif choice == "3":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            
            img_path = input("Görüntü yolu girin: ").strip()
            if os.path.exists(img_path):
                detector.visualize_predictions(img_path)
            else:
                print("❌ Dosya bulunamadı!")
                
        elif choice == "4":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
                
            folder_path = input("Klasör yolu girin: ").strip()
            if os.path.exists(folder_path):
                detector.predict_batch(folder_path)
            else:
                print("❌ Klasör bulunamadı!")
                
        elif choice == "5":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.evaluate_model()
            
        elif choice == "6":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.create_detection_report()
            
        elif choice == "7":
            if detector.model is None:
                print("❌ Önce model yükleyin!")
                continue
            detector.live_detection_demo()
            
        elif choice == "0":
            print("👋 Görüşmek üzere!")
            break
            
        else:
            print("❌ Geçersiz seçim!")

if __name__ == "__main__":
    main()

📁 Veri seti yapısı kontrol ediliyor...
Dataset yolu: D:\NOT\sixray_yolov8.v964i.yolov8
✅ YAML config dosyası bulundu: data.yaml
✅ Train: 5806 görüntü, 5806 etiket
✅ Valid: 1628 görüntü, 1628 etiket
✅ Test: 829 görüntü, 829 etiket
🎯 SixRay YOLOv8 Object Detection

📋 Menü:
1. Model eğit
2. Eğitilmiş model yükle
3. Tek görüntü test et
4. Klasör test et
5. Model değerlendir
6. Test raporu oluştur
7. Canlı detection demo
0. Çıkış



Seçiminiz (0-7):  1
Epoch sayısı (varsayılan 100):  200
Batch size (varsayılan 16):  32
Model boyutu (n/s/m/l/x, varsayılan n):  n


✅ yolov8n.pt modeli yüklendi
📊 Device: CUDA
🚀 Model eğitimi başlıyor...
   Epochs: 200
   Batch Size: 32
   Image Size: 640
Ultralytics 8.3.156  Python-3.10.5 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\NOT\sixray_yolov8.v964i.yolov8\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=sixra


train: Scanning D:\NOT\sixray_yolov8.v964i.yolov8\train\labels.cache... 5806 images, 0 backgrounds, 0 corrupt: 100%|███

WARNING train: 10.0GB RAM required to cache images with 50% safety margin but only 4.0/15.9GB available, not caching images
val: Fast image access  (ping: 0.20.1 ms, read: 1.00.5 MB/s, size: 38.8 KB)


...

📋 Detaylı test raporu oluşturuluyor...

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00031_jpg.rf.9018d98e3267ad29b3ab760f50df409e.jpg: 640x640 2 Guns, 1 Knife, 70.3ms
Speed: 29.4ms preprocess, 70.3ms inference, 98.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00060_jpg.rf.7b78e6a9d19187626b8b442b0bf46f24.jpg: 640x640 2 Guns, 2 Knifes, 47.6ms
Speed: 4.8ms preprocess, 47.6ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00061_jpg.rf.11aa3ce8671fafbc49a1c29ee5c5ca0f.jpg: 640x640 1 Knife, 89.9ms
Speed: 4.7ms preprocess, 89.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00082_jpg.rf.c162d183064b829fbdb5e1d4a9ef386d.jpg: 640x640 1 Gun, 1 Knife, 10.5ms
Speed: 3.0ms preprocess, 10.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\N


Seçiminiz (0-7):  6


📋 Detaylı test raporu oluşturuluyor...

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00031_jpg.rf.9018d98e3267ad29b3ab760f50df409e.jpg: 640x640 2 Guns, 1 Knife, 61.5ms
Speed: 31.6ms preprocess, 61.5ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00060_jpg.rf.7b78e6a9d19187626b8b442b0bf46f24.jpg: 640x640 2 Guns, 2 Knifes, 59.5ms
Speed: 24.7ms preprocess, 59.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00061_jpg.rf.11aa3ce8671fafbc49a1c29ee5c5ca0f.jpg: 640x640 1 Knife, 64.9ms
Speed: 4.5ms preprocess, 64.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\NOT\sixray_yolov8.v964i.yolov8\test\images\P00082_jpg.rf.c162d183064b829fbdb5e1d4a9ef386d.jpg: 640x640 1 Gun, 1 Knife, 61.0ms
Speed: 13.8ms preprocess, 61.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\


Seçiminiz (0-7):  0


👋 Görüşmek üzere!
